In [1]:
import wandb
import os
import numpy as np
from datasets import load_dataset
from transformers import Seq2SeqTrainingArguments as TrainingArguments
from transformers import Seq2SeqTrainer as Trainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import evaluate
import datetime

In [2]:
source_lang = "zh"
target_lang = "en"

def preprocess_function(examples):
    inputs = [example[source_lang] for example in examples["translation"]]
    targets = [example[target_lang] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, padding="max_length", max_length=32, truncation=True)
    model_inputs["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in model_inputs["labels"]]
    return model_inputs

metric = evaluate.load("sacrebleu")
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# download prepare the data
dataset = load_dataset("iwslt2017", "iwslt2017-zh-en", cache_dir="./cache") # optional
checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenized_sentences = dataset.map(preprocess_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
c:\Users\xk_20\Documents\Code\.TFT_OCR_BOT\lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.config.max_length = 32
model.config.min_length = 8
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4


dt = datetime.datetime.now(datetime.timezone.utc)
dt = dt.replace(microsecond=0, tzinfo=None)

# set the wandb project where this run will be logged
os.environ["WANDB_PROJECT"]="MT5"
# save your trained model checkpoint to wandb
os.environ["WANDB_LOG_MODEL"]="true"
# turn off watch to log faster
os.environ["WANDB_WATCH"]="false"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"
os.environ["WANDB_NAME"] = str(dt)

# pass "wandb" to the 'report_to' parameter to turn on wandb logging
training_args = TrainingArguments(
    output_dir='wandb_mt5',
    report_to="wandb",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="steps",
    logging_steps=2000,
    eval_steps= 2000,
    max_steps = 300000,
    save_strategy="steps",
    save_steps = 5000,
    learning_rate=1e-5,
    weight_decay=0.005,
    predict_with_generate=True,
)

# define the trainer and start training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_sentences["train"],
    eval_dataset=tokenized_sentences["validation"],
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train(resume_from_checkpoint="./wandb_mt5/checkpoint-100000")

# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


  0%|          | 0/300000 [00:00<?, ?it/s]

c:\Users\xk_20\Documents\Code\.TFT_OCR_BOT\lib\site-packages\transformers\generation\utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  0%|          | 0/55 [00:00<?, ?it/s]

{'eval_loss': 2.1504266262054443, 'eval_bleu': 13.223, 'eval_gen_len': 23.8589, 'eval_runtime': 59.9964, 'eval_samples_per_second': 14.651, 'eval_steps_per_second': 0.917, 'epoch': 6.93}


...

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 32, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


{'eval_loss': 2.2659811973571777, 'eval_bleu': 14.1212, 'eval_gen_len': 23.694, 'eval_runtime': 46.3468, 'eval_samples_per_second': 18.966, 'eval_steps_per_second': 1.187, 'epoch': 20.75}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 32, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


{'train_runtime': 92143.844, 'train_samples_per_second': 52.092, 'train_steps_per_second': 3.256, 'train_loss': 0.9730732552083333, 'epoch': 20.75}


eval/bleu,▁▃▂▅▅▄▄▄▄▄▄▆▆▆▆▅▅█▆▅▆▅▅█▇█▆█▆▇█▇▇▇▇▇▇▇▇▇
eval/gen_len,▂▄█▄▆▆▅▅▅▄▇█▆▄▃▅▃▃▃▅▅▁▂▃▂▃▂▂▅▄▃▄▄▁▃▄▄▂▃▂
eval/loss,▅▄▃▄▃▂▄▂▂▂▂▁▃▂▁▃▂▂▃▂▃▄▃▅▄▃▆▅▄▆▆▅▇▇▆▇▇▇██
eval/runtime,▁▁▇▇▇▇▇▇▇▇█▇▇▇▇▇▁▁▁▁█▁▂▄▁▂▁▁▁▁▁▁▄▆█▁▁▁▁▁
eval/samples_per_second,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁████▁█▇▄█▆█▇▇███▄▂▁████▇
eval/steps_per_second,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁████▁█▇▄█▆█▇▇███▄▂▁████▇
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
train/grad_norm,▆▁▂▆▂▅▄▂▁▂▂█▅▅▁▄▅▂▃▄▄▄▁▄▄▅▁▂▁▁▂▆▁▂▃▃▄▅▆▃
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▇██▇▇▇▆▆▆▅▆▆▅▅▅▄▄▅▄▄▄▃▃▃▃▃▂▂▃▂▂▂▂▂▂▁▁▁▁▁


In [11]:
path = "./wandb_mt5/mt5-tuned"
trainer.save_model(path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 32, 'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


In [17]:
text = "我喜欢吃鸡饭和冰淇淋。"

In [18]:
from transformers import pipeline

model = AutoModelForSeq2SeqLM.from_pretrained("./wandb_mt5/mt5-tuned")
translator = pipeline("translation", model=model, tokenizer=tokenizer)
translator(text)

[{'translation_text': 'I liked chicken and ice cream.'}]